In [4]:
import pandas as pd

Import all the files

In [3]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

- visits lists all of the users who have visited the website
- cart lists all of the users who have added a t-shirt to their cart
- checkout lists all of the users who have started the checkout
- purchase lists all of the users who have purchased a t-shirt

In [5]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [11]:
visits_cart = pd.merge(
    visits
    , cart
    , how = 'left'
)
print(visits_cart.head(3))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  


Step 3: How long is `visits_cart`?

In [12]:
len(visits_cart) # which is the same as the length of visits (left join)

2000

Step 4: How many timestamps are null for `cart_time`?

In [18]:
not_null_carts = pd.notnull(visits_cart.cart_time).sum()
not_null_carts

348

Step 5: What percentage only visited?

In [20]:
total_visits = len(visits_cart)
perc_only_visited = not_null_carts/total_visits
print("{} only visited the page".format(perc_only_visited)) 

0.174 only visited the page


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [25]:
visits_cart_checkout = pd.merge(
    visits_cart
    , checkout
    , how = 'left'
)
print(visits_cart_checkout.head(3))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   

            cart_time       checkout_time  
0                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00  
2 2017-08-20 08:31:00                 NaT  


In [31]:
#cheking results
not_null_checkout = pd.notnull(visits_cart_checkout.checkout_time).sum()
print(not_null_checkout)


226


Step 7: Merge it all together

In [33]:
funnel = pd.merge(
    visits_cart_checkout
    , purchase
    , how = 'left'
)
print(funnel.head(3))

visits = pd.notnull(funnel.visit_time).sum()
carts = pd.notnull(funnel.cart_time).sum()
checkouts = pd.notnull(funnel.checkout_time).sum()
purchases = pd.notnull(funnel.purchase_time).sum()

#Printing the funnel values
print("Funnel: Visits = {} -> Carts = {} -> Checkout = {} -> Purchase = {} ".format(visits, carts, checkouts, purchases) )


                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
Funnel: Visits = 2108 -> Carts = 456 -> Checkout = 334 -> Purchase = 252 


Step 8: % of users who got to checkout but did not purchase

In [34]:
1-purchases/checkouts

0.2455089820359282

Step 9: check each part of the funnel, let's print all 3 of them again

In [40]:
perc_to_cart = round(carts/visits,2)
perc_to_checkout = round(checkouts/carts,2)
perc_to_purchase = round(purchases/checkouts,2)

print("Funnel: \n")
print("{}% converted from Visits to Carts".format(perc_to_cart*100))
print("{}% converted from Carts to Checkouts".format(perc_to_checkout*100))
print("{}% converted from Checkouts to Purchases".format(perc_to_purchase*100))


Funnel: 

22.0% converted from Visits to Carts
73.0% converted from Carts to Checkouts
75.0% converted from Checkouts to Purchases


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [42]:
# Let’s calculate the average time
funnel['time_to_buy'] = funnel.purchase_time - funnel.visit_time


                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time     time_to_buy  
0                 NaT                 NaT                 NaT             NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00 0 days 00:44:00  
2 2017-08-20 08:31:00                 NaT                 NaT             NaT  
3                 NaT                 NaT                 NaT             NaT  
4                 NaT                 NaT                 NaT             NaT  


Step 11: examine the results

In [48]:
funnel_just_purchases = funnel[funnel.time_to_buy.notnull()]
print(funnel_just_purchases.head(10))
# here we can see time_to_buy is correct taking a few examples

                                 user_id          visit_time  \
1   0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
14  486480e2-98c3-4d51-8f4b-b1c07228ce84 2017-01-27 16:34:00   
48  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
49  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
65  ab0125fc-9493-4f59-ad70-24ad264a3a0c 2017-11-18 03:21:00   
70  ba65fa41-b88f-4ebc-8e66-7ade432f4f0d 2017-05-14 11:01:00   
71  ba65fa41-b88f-4ebc-8e66-7ade432f4f0d 2017-05-14 11:01:00   
72  ba65fa41-b88f-4ebc-8e66-7ade432f4f0d 2017-05-14 11:01:00   
75  1f117a3f-5b5e-42ab-9dd6-42ade85771d2 2017-08-13 09:18:00   
76  1f117a3f-5b5e-42ab-9dd6-42ade85771d2 2017-08-13 09:18:00   

             cart_time       checkout_time       purchase_time     time_to_buy  
1  2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00 0 days 00:44:00  
14 2017-01-27 16:44:00 2017-01-27 17:10:00 2017-01-27 17:12:00 0 days 00:38:00  
48 2017-01-08 20:38:00 2017-01-08 20:52:00 2017-01-0

Step 12: average time to purchase